<a href="https://colab.research.google.com/github/antoniomuso/speech2face/blob/master/face_processing_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip3 install face_recognition
! pip install deepface

In [ ]:
import glob

import numpy as np
import sys

from PIL import Image
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import  torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from face_recognition import face_landmarks, face_locations, face_encodings
from random import randint

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim

import os
from os.path import join

from tqdm import tqdm
from google.colab.patches import cv2_imshow
import cv2
from deepface import DeepFace
import sys

from os.path import join
from os import listdir
import random
from IPython.utils import io
import tensorflow as tf

tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
 ! cp "/content/drive/My Drive/Speech2Face/zippedFaces.tar.gz" /content
 ! tar zxvf zippedFaces.tar.gz
 ! rm zippedFaces.tar.gz


In [ ]:
class Vgg_face_dag(nn.Module):

    def __init__(self):
        super(Vgg_face_dag, self).__init__()
        self.meta = {'mean': [129.186279296875, 104.76238250732422, 93.59396362304688],
                     'std': [1, 1, 1],
                     'imageSize': [224, 224, 3]}
        self.conv1_1 = nn.Conv2d(3, 64, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1))
        self.relu1_1 = nn.ReLU(inplace=True)
        self.conv1_2 = nn.Conv2d(64, 64, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1))
        self.relu1_2 = nn.ReLU(inplace=True)
        self.pool1 = nn.MaxPool2d(kernel_size=[2, 2], stride=[2, 2], padding=0, dilation=1, ceil_mode=False)
        self.conv2_1 = nn.Conv2d(64, 128, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1))
        self.relu2_1 = nn.ReLU(inplace=True)
        self.conv2_2 = nn.Conv2d(128, 128, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1))
        self.relu2_2 = nn.ReLU(inplace=True)
        self.pool2 = nn.MaxPool2d(kernel_size=[2, 2], stride=[2, 2], padding=0, dilation=1, ceil_mode=False)
        self.conv3_1 = nn.Conv2d(128, 256, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1))
        self.relu3_1 = nn.ReLU(inplace=True)
        self.conv3_2 = nn.Conv2d(256, 256, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1))
        self.relu3_2 = nn.ReLU(inplace=True)
        self.conv3_3 = nn.Conv2d(256, 256, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1))
        self.relu3_3 = nn.ReLU(inplace=True)
        self.pool3 = nn.MaxPool2d(kernel_size=[2, 2], stride=[2, 2], padding=0, dilation=1, ceil_mode=False)
        self.conv4_1 = nn.Conv2d(256, 512, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1))
        self.relu4_1 = nn.ReLU(inplace=True)
        self.conv4_2 = nn.Conv2d(512, 512, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1))
        self.relu4_2 = nn.ReLU(inplace=True)
        self.conv4_3 = nn.Conv2d(512, 512, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1))
        self.relu4_3 = nn.ReLU(inplace=True)
        self.pool4 = nn.MaxPool2d(kernel_size=[2, 2], stride=[2, 2], padding=0, dilation=1, ceil_mode=False)
        self.conv5_1 = nn.Conv2d(512, 512, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1))
        self.relu5_1 = nn.ReLU(inplace=True)
        self.conv5_2 = nn.Conv2d(512, 512, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1))
        self.relu5_2 = nn.ReLU(inplace=True)
        self.conv5_3 = nn.Conv2d(512, 512, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1))
        self.relu5_3 = nn.ReLU(inplace=True)
        self.pool5 = nn.MaxPool2d(kernel_size=[2, 2], stride=[2, 2], padding=0, dilation=1, ceil_mode=False)
        self.fc6 = nn.Linear(in_features=25088, out_features=4096, bias=True)
        self.relu6 = nn.ReLU(inplace=True)
        self.dropout6 = nn.Dropout(p=0.5)
        self.fc7 = nn.Linear(in_features=4096, out_features=4096, bias=True)
        self.relu7 = nn.ReLU(inplace=True)
        self.dropout7 = nn.Dropout(p=0.5)
        self.fc8 = nn.Linear(in_features=4096, out_features=2622, bias=True)

    def forward(self, x0, is_fc8=False):
        x1 = self.conv1_1(x0)
        x2 = self.relu1_1(x1)
        x3 = self.conv1_2(x2)
        x4 = self.relu1_2(x3)
        x5 = self.pool1(x4)
        x6 = self.conv2_1(x5)
        x7 = self.relu2_1(x6)
        x8 = self.conv2_2(x7)
        x9 = self.relu2_2(x8)
        x10 = self.pool2(x9)
        x11 = self.conv3_1(x10)
        x12 = self.relu3_1(x11)
        x13 = self.conv3_2(x12)
        x14 = self.relu3_2(x13)
        x15 = self.conv3_3(x14)
        x16 = self.relu3_3(x15)
        x17 = self.pool3(x16)
        x18 = self.conv4_1(x17)
        x19 = self.relu4_1(x18)
        x20 = self.conv4_2(x19)
        x21 = self.relu4_2(x20)
        x22 = self.conv4_3(x21)
        x23 = self.relu4_3(x22)
        x24 = self.pool4(x23)
        x25 = self.conv5_1(x24)
        x26 = self.relu5_1(x25)
        x27 = self.conv5_2(x26)
        x28 = self.relu5_2(x27)
        x29 = self.conv5_3(x28)
        x30 = self.relu5_3(x29)
        x31_preflatten = self.pool5(x30)
        x31 = x31_preflatten.view(x31_preflatten.size(0), -1)
        x32 = self.fc6(x31)
        x33 = self.relu6(x32)
        x34 = self.dropout6(x33)
        x35 = self.fc7(x34)
        x36 = self.relu7(x35)
        x37 = self.dropout7(x36)
        if is_fc8:
            x38 = self.fc8(x37)
        else:
            x38 = x37
        return x38


def vgg_face_dag(weights_path=None, **kwargs):
    """
    load imported model instance
    Args:
        weights_path (str): If set, loads model weights from the given path
    """
    model = Vgg_face_dag()
    if weights_path:
        state_dict = torch.load(weights_path)
        model.load_state_dict(state_dict)
    return model

In [ ]:
class _Normalize_Tensor(object):
    def __init__(self, color_mean, color_std):
        self.color_mean = color_mean
        self.color_std = color_std

    def __call__(self, img):
        # Convert image to Tensor
        img = transforms.functional.to_tensor(img)

        # Normalize image by the parameter of pre-trained face-encoder
        img = transforms.functional.normalize(
            img, self.color_mean, self.color_std)

        return img

In [ ]:
import os, sys

class HiddenPrints:
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')

    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stdout = self._original_stdout

In [ ]:
datasetPath = 'unzippedFaces'
base_dir='/content/drive/My Drive/Speech2Face/'
data_points = os.listdir(datasetPath) ## GETTING DATA POINTS THAT IS NAME OF CELEBRTIES.

In [ ]:
import gc

tot_deleted = 0
tot_images = 0
for i in tqdm(range(len(data_points))):
  dir = data_points[i]

  #for file_name in to_delete:
  #  os.remove(file_name)
  #to_delete = [] 
  s = join(datasetPath, dir+"/1.6/")
  for subdir in listdir(s):
    #print(subdir)
    t = join(s, subdir)
    for image in listdir(t):
      gc.collect()
      image_path = join(t, image)
      #print(image_path)

      tot_images += 1
      #print(full_path)

      try:
          with io.capture_output() as captured:
            out = DeepFace.analyze(image_path, actions=['emotion'])
            DeepFace.
        
      except:
        os.remove(image_path)
        #print("removed", image_path)
        tot_deleted += 1
        continue
      
      if (out['dominant_emotion'] != 'neutral'):
        os.remove(image_path)
        tot_deleted += 1


print("Total images:", tot_images)
print("Total_deleted:", tot_deleted)





In [ ]:
device = 'cuda'
face_encoder_path = join(base_dir, "Pretrained", "vgg_face_dag.pth")
face_encoder_model = vgg_face_dag(face_encoder_path)
face_encoder_model.eval()
face_encoder_model.to(device)

color_mean, color_std = face_encoder_model.meta["mean"], face_encoder_model.meta["std"]
color_mean = [tmp / 255.0 for tmp in color_mean]
color_std = [tmp / 255.0 for tmp in color_std]

transform_fe = transforms.Compose([
            transforms.Resize((224, 224)),
            _Normalize_Tensor(color_mean, color_std)
])



In [ ]:
no_of_record_per_image = 3  ## NUMBER OF TIMES EVERY CELEBRITIES (DIFFERENT) IMAGES IS TO BE STORED
face_feature = np.zeros((len(data_points)*no_of_record_per_image,4096)) ### ARRAY THAT WILL STORE THE FACE FEATURE
if not("Face_Feature" in os.listdir()): ### MAKING DIR
    os.mkdir("Face_Feature")
    os.mkdir("Face_Feature/Faces")
index = 0
### EXTRACTING FACE FEATURE.
for i in tqdm(range(len(data_points))):
    for j in range(no_of_record_per_image):
      try: 
        path_img = join(datasetPath,data_points[i],"1.6",random.choice(listdir(join(datasetPath,data_points[i],"1.6"))))
        image_path = join(path_img ,  random.choice(listdir(path_img)))
        img = Image.open(image_path)
      except:
        continue

#      try:
#          with io.capture_output() as captured:
#            out = DeepFace.analyze(image_path, actions=['emotion'])
#        
#      except:
#        continue
#      
#      if (out['dominant_emotion'] != 'neutral'):
#       continue

      # pos = face_locations(np.array(img), model='cnn')

      # if len(pos) == 0:
      #  continue   

      encoding = face_encoder_model(transform_fe(img).unsqueeze_(0).to(device))
      # face_feature[index] = encoding.cpu().detach().numpy()
      face_feature[index,:] = encoding.cpu().detach().numpy()

      # print(face_feature[index].sum())
      filename = data_points[i] + "_"  + str(j) + ".jpg"
      path_save = join("Face_Feature/Faces", filename)

      img.save(path_save)
    
      index += 1
      
    
np.save("Face_Feature/facefeature.npy",face_feature[:index])
print(index)

In [ ]:
 # np.save("Face_Feature/facefeature.npy",face_feature[:index  ])
 print(face_feature[1].sum())
 print(face_feature.shape)

In [ ]:
# Save as in the repo

np.save("facefeature_save2_filtered.npy",face_feature)
!cp facefeature_save2.npy /content/drive/My\ Drive/Speech2Face/

In [ ]:
# prova = np.load("Face_Feature/facefeature.npy")
# print(prova[3752].sum())

! zip -r face_features_neutral_prefiltered.zip Face_Feature/ 
! cp face_features_neutral_prefiltered.zip /content/drive/My\ Drive/Speech2Face/
! rm -r Face_Feature/